In [1]:
library(devtools)
library(tidyverse)
library(cicero)

Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.1          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: monocle

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are maske

In [2]:
peaks_intersections <- read.table("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/02.peak_accessability/uf.peaks_accessability.txt")
all_peaks <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/03.peak_assignment/uf_dir/uf_acr_classification.no_exons.all_ACRs.classified.sorted.bed", delim = "\t",
                          col_names = c("chrom", "start", "stop", "acr_class", "pval"))

read_meta_data <- read.delim("/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/annotations/uf_annot/uf.leaf_annotation.V1.meta.txt") 
genome <- read.table("/home/jpm73279/genome_downloads/Ufusca/v1.1/assembly/Ufusca_669_v1.0.Pt.sizes")




generate_cicero_format <- all_peaks %>% 
    dplyr::mutate(combined_vals = str_c(chrom, start,stop, sep = "_")) %>% 
    tidyr::separate(acr_class, c("ACR", "category"), ";")


final_cicero_format <- left_join(peaks_intersections, generate_cicero_format, by = c("V1" = "ACR")) %>% 
              dplyr::select(combined_vals, V2, V3)


reduced_dim <- read_meta_data %>% 
    dplyr::select(cellID, umap1, umap2) %>% 
    dplyr::rename(umap_coord1 = umap1) %>% 
    dplyr::rename(umap_coord2 = umap2)

row.names(reduced_dim) <- reduced_dim$cellID
reduced_dim <- reduced_dim[-1]

final_cicero_format_filtered <- final_cicero_format %>% 
    dplyr::filter(V2 %in% read_meta_data$cellID)


Rows: 29153 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chrom, acr_class, pval
dbl (2): start, stop

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [8]:
colnames(read_meta_data)

[1] "cellID"               "total"                "tss"                 
 [4] "acrs"                 "ptmt"                 "nSites"              
 [7] "log10nSites"          "pTSS"                 "FRiP"                
[10] "pOrg"                 "tss_z"                "acr_z"               
[13] "sites_z"              "qc_check"             "background"          
[16] "cellbulk"             "is_cell"              "sampleID"            
[19] "umap1"                "umap2"                "doubletscore"        
[22] "d.type"               "LouvainClusters"      "Louvain_cluster_safe"
[25] "annotation_v1"        "subcluster_lc_safe"   "umap1.subcluster"    
[28] "umap2.subcluster"     "counts"               "annotation_v1_ncell"

In [3]:
setwd("/scratch/jpm73279/comparative_single_cell/dev_location/cicero")

In [9]:

for(COL_SLOT_ID in unique(read_meta_data$annotation_v1)) {
  
  # Subset meta data based on the unique value of COLUMN_NAME
  subset_meta_data <- read_meta_data[read_meta_data$annotation_v1 == COL_SLOT_ID, ]
  print(COL_SLOT_ID)
  # Update reduced_dim based on subset
  reduced_dim <- subset_meta_data %>% 
    dplyr::select(cellID, umap1, umap2) %>% 
    dplyr::rename(umap_coord1 = umap1) %>% 
    dplyr::rename(umap_coord2 = umap2)

  row.names(reduced_dim) <- reduced_dim$cellID
  reduced_dim <- reduced_dim[-1]
  
  # Update final_cicero_format_filtered based on subset
  final_cicero_format_filtered <- final_cicero_format %>% 
    dplyr::filter(V2 %in% subset_meta_data$cellID)
  
  input_cds <- make_atac_cds(final_cicero_format_filtered, binarize = TRUE)
  
  # Make cicero CDS
  cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = reduced_dim)
  
  # Run cicero and save output
  running_cic <- run_cicero(cicero_cds, genome)
  write.csv(running_cic, paste0("uf.cicero.", COL_SLOT_ID, ".output.csv"))
}

[1] "mesophyll"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.511985791798242
Median shared cells bin-bin: 0

Warning message in if (isSparseMatrix(counts)) {:
"the condition has length > 1 and only the first element will be used"


[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  1239"
[1] "Other models: "

Zero or one element in range 
                          28 
[1] "Models with errors:  0"
[1] "Done"
[1] "epidermis"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.468870902000959
Median shared cells bin-bin: 0

Warning message in if (isSparseMatrix(counts)) {:
"the condition has length > 1 and only the first element will be used"


[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  1239"
[1] "Other models: "

Zero or one element in range 
                          28 
[1] "Models with errors:  0"
[1] "Done"
[1] "bundle_sheath"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.357687032019822
Median shared cells bin-bin: 0

Warning message in if (isSparseMatrix(counts)) {:
"the condition has length > 1 and only the first element will be used"


[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  1239"
[1] "Other models: "

Zero or one element in range 
                          28 
[1] "Models with errors:  0"
[1] "Done"
[1] "companion_cells_sieve_elements"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 2.24625391430371
Median shared cells bin-bin: 0

Warning message in if (isSparseMatrix(counts)) {:
"the condition has length > 1 and only the first element will be used"


[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  1239"
[1] "Other models: "

Zero or one element in range 
                          28 
[1] "Models with errors:  0"
[1] "Done"


In [5]:
running_cic %>% 
    dplyr::filter(coaccess > 0)

ERROR: Error in dplyr::filter(., coaccess > 0): object 'running_cic' not found
